## 肺癌のセグメンテーションと良悪性の判別モデル

１．CTデータをpytorchで扱えるように

２．肺の腫瘍のみをセグメンテーション（タスクの焦点を絞るために）

３．関心ボクセルをグループ化して結節候補をまとめる

４．結節候補をConvalutonで分類する

５．結節ごとを患者ごとへの評価にする（今回は悪性度の最大値）

こぶの内悪性が結節
annotationsは結節フラグの大きさ
candidatesはすべてのこぶ情報

## annotaitonとcandidateで位置の統合

In [2]:
from collections import namedtuple
CandidateInfoTuple = namedtuple(
    'CandidateInfoTuple',
    'isNodule_bool, diameter_mm, series_uid, center_xyz',
)

In [3]:
CandidateInfoTuple

__main__.CandidateInfoTuple

In [73]:
#CTデータの内mhdファイルをリストにする
#メモリに残す関数:lru_cache

import functools
import glob
import os

#mhd_list = glob.glob('data-unversioned/part2/luna/subset*/*.mhd')
mhd_list = glob.glob('../deep-learning-with-pytorch-ja/data/part2/luna/subset*/*.mhd')

presentOnDisk_set = {os.path.split(p)[-1][:-4] for p in mhd_list}

In [74]:
presentOnDisk_set

{'1.3.6.1.4.1.14519.5.2.1.6279.6001.105756658031515062000744821260',
 '1.3.6.1.4.1.14519.5.2.1.6279.6001.108197895896446896160048741492',
 '1.3.6.1.4.1.14519.5.2.1.6279.6001.109002525524522225658609808059'}

In [47]:
import csv


diameter_dict = {}
with open('../deep-learning-with-pytorch-ja/data/part2/luna/annotations.csv', "r") as f:
        for row in list(csv.reader(f))[1:]:
            series_uid = row[0]
            annotationCenter_xyz = tuple([float(x) for x in row[1:4]])
            annotationDiameter_mm = float(row[4])

            diameter_dict.setdefault(series_uid, []).append(
                (annotationCenter_xyz, annotationDiameter_mm)
            )

In [16]:
#diameter_dict

#uid一つに複数のこぶがある場合もあり、こぶの中心座標と大きさがvalueに入る。

In [30]:
a="1.3.6.1.4.1.14519.5.2.1.6279.6001.100621383016233746780170740405"

In [38]:
#辞書のaの要素を返す。なければ台に引数を返す。デフォはNone
b=diameter_dict.get(a,[])
print(b)
s=diameter_dict.get("lll",[])
print(s)
f=diameter_dict.get("lll")
print(f)


[((-24.0138242, 192.1024053, -391.0812764), 8.143261683), ((2.441546798, 172.4648812, -405.4937318), 18.54514997), ((90.93171321, 149.0272657, -426.5447146), 18.20857028), ((89.54076865, 196.4051593, -515.0733216), 16.38127631)]
[]
None


In [64]:
#sereis_uidでまわして、個々の結節の直径のデータごとに回して、ｘ、ｙ、ｚの各軸ごとにまわす

candidateInfo_list = []
with open('../deep-learning-with-pytorch-ja/data/part2/luna/candidates.csv', "r") as f:
    for row in list(csv.reader(f))[1:]:
        series_uid = row[0]
#presentOnDisk_setはCTデータのuid
#そこに無いのはスキップすることで直径０として使う
        if series_uid not in presentOnDisk_set :
            continue

        isNodule_bool = bool(int(row[4]))
        candidateCenter_xyz = tuple([float(x) for x in row[1:4]])

        candidateDiameter_mm = 0.0
        for annotation_tup in diameter_dict.get(series_uid, []):
            annotationCenter_xyz, annotationDiameter_mm = annotation_tup
            for i in range(3):
                #距離（差）が大きいものは確認(直径/2/2との比較)
                delta_mm = abs(candidateCenter_xyz[i] - annotationCenter_xyz[i])
                if delta_mm > annotationDiameter_mm / 4:
                    break
            else:
                # 差が小さいものは統一する
                candidateDiameter_mm = annotationDiameter_mm
                break

        candidateInfo_list.append(CandidateInfoTuple(
            isNodule_bool,
            candidateDiameter_mm,
            series_uid,
            candidateCenter_xyz,
        ))

In [65]:
candidateInfo_list[0:5]

[CandidateInfoTuple(isNodule_bool=False, diameter_mm=0.0, series_uid='1.3.6.1.4.1.14519.5.2.1.6279.6001.105756658031515062000744821260', center_xyz=(129.56815129, 45.3770967403, -277.835757804)),
 CandidateInfoTuple(isNodule_bool=False, diameter_mm=0.0, series_uid='1.3.6.1.4.1.14519.5.2.1.6279.6001.105756658031515062000744821260', center_xyz=(-97.26, 56.36, -201.93)),
 CandidateInfoTuple(isNodule_bool=False, diameter_mm=0.0, series_uid='1.3.6.1.4.1.14519.5.2.1.6279.6001.105756658031515062000744821260', center_xyz=(99.2304910833, -4.88216543725, -128.691304564)),
 CandidateInfoTuple(isNodule_bool=False, diameter_mm=0.0, series_uid='1.3.6.1.4.1.14519.5.2.1.6279.6001.105756658031515062000744821260', center_xyz=(-15.287525202, -59.1051002263, -251.303750179)),
 CandidateInfoTuple(isNodule_bool=False, diameter_mm=0.0, series_uid='1.3.6.1.4.1.14519.5.2.1.6279.6001.105756658031515062000744821260', center_xyz=(109.344666074, 1.89173714531, -237.284698998))]

In [66]:
candidateInfo_list.sort(reverse=True)

In [69]:
candidateInfo_list[1:5]

[CandidateInfoTuple(isNodule_bool=True, diameter_mm=6.440878725, series_uid='1.3.6.1.4.1.14519.5.2.1.6279.6001.108197895896446896160048741492', center_xyz=(-100.46, 68.01, -230.55)),
 CandidateInfoTuple(isNodule_bool=True, diameter_mm=0.0, series_uid='1.3.6.1.4.1.14519.5.2.1.6279.6001.109002525524522225658609808059', center_xyz=(44.25, 52.17, -110.25)),
 CandidateInfoTuple(isNodule_bool=True, diameter_mm=0.0, series_uid='1.3.6.1.4.1.14519.5.2.1.6279.6001.109002525524522225658609808059', center_xyz=(36.54, 78.1, -122.92)),
 CandidateInfoTuple(isNodule_bool=False, diameter_mm=0.0, series_uid='1.3.6.1.4.1.14519.5.2.1.6279.6001.109002525524522225658609808059', center_xyz=(49.0692129504, 23.0584814498, -98.6156878657))]

## CTデータ取込み

In [71]:
import SimpleITK as sitk
import numpy as np

In [1]:
from util.util import XyzTuple, xyz2irc


#series_uidごとにパスをつくり
mhd_path = glob.glob(
    '../deep-learning-with-pytorch-ja/data/part2/luna/subset*/*.mhd'.format(series_uid))[0]
#CTを取り込む
ct_mhd = sitk.ReadImage(mhd_path)
ct_a = np.array(sitk.GetArrayFromImage(ct_mhd), dtype=np.float32)

#CT値をこの範囲のみに絞るそれ以外は今回の予測には不要と判断
#新たなデータが追加された時もこの処理をしたことを覚えておく必要がある。
ct_a.clip(-1000, 1000, ct_a)

#self.series_uid = series_uid
#self.hu_a = ct_a

NameError: name 'glob' is not defined

ディープラーニングは固定長であるため、入力も固定長でないといけない。

## 位置座標の修正

データではmm,欲しいデータはボクセルの位置情報であるため、修正が必要

TrasformIndexToPhysicalPoint
TranformPhysicalPointToIndex
では画像データの読み込んで保持した後に変換する

保持しないで変換すすため自作している。IRCからXYZ

1、XYZに合わせるために座標をIRCからCRIに反転する

2、ボクセルサイズに合わせてインデックスをスケーリングする

3、pythonの＠で行列の積を行う

4、原点に対してオフセットを加える

In [12]:
# collectionの確認

from collections import namedtuple
N=namedtuple('Fruit',['apple','grape'])
N(*[1,2])  #　リストを展開

Fruit(apple=1, grape=2)

In [13]:
n=N(*[1,2])
n.apple

1

In [58]:
import numpy as np
a=np.zeros([3,3,4])
a[0:2]=1
a[1]=2
a

array([[[1., 1., 1., 1.],
        [1., 1., 1., 1.],
        [1., 1., 1., 1.]],

       [[2., 2., 2., 2.],
        [2., 2., 2., 2.],
        [2., 2., 2., 2.]],

       [[0., 0., 0., 0.],
        [0., 0., 0., 0.],
        [0., 0., 0., 0.]]])

In [64]:
b=a[::-1]
b

array([[[0., 0., 0., 0.],
        [0., 0., 0., 0.],
        [0., 0., 0., 0.]],

       [[2., 2., 2., 2.],
        [2., 2., 2., 2.],
        [2., 2., 2., 2.]],

       [[1., 1., 1., 1.],
        [1., 1., 1., 1.],
        [1., 1., 1., 1.]]])

In [79]:
import collections
IrcTuple = collections.namedtuple('IrcTuple', ['index', 'row', 'col'])
XyzTuple = collections.namedtuple('XyzTuple', ['x', 'y', 'z'])

In [88]:
#XyzTupleに三つ入れたらそれぞれx、y、zでそれぞれの要素を取り出せる
a=XyzTuple(*["x座標",'y座標','z座標'])
print(type(a))
a.x

<class '__main__.XyzTuple'>


'x座標'

In [89]:
for d, f in enumerate(a):
    print(d)
    print(f)

0
x座標
1
y座標
2
z座標


In [76]:
#CTデータから位置情報を取り出す
origin_xyz = XyzTuple(*ct_mhd.GetOrigin())
vxSize_xyz = XyzTuple(*ct_mhd.GetSpacing())
direction_a = np.array(ct_mhd.GetDirection()).reshape(3, 3)


#原点 = (-127、-96、-110)
#ボクセルサイズ
#方向 = 恒等行列



NameError: name 'ct_mhd' is not defined

In [ ]:
# 結節の中心のデータ（mm)、原点、ボクセルサイズ、方向が引数
def xyz2irc(coord_xyz, origin_xyz, vxSize_xyz, direction_a):
    origin_a = np.array(origin_xyz)
    vxSize_a = np.array(vxSize_xyz)
    coord_a = np.array(coord_xyz)
    cri_a = ((coord_a - origin_a) @ np.linalg.inv(direction_a)) / vxSize_a
    cri_a = np.round(cri_a)
    return IrcTuple(int(cri_a[2]), int(cri_a[1]), int(cri_a[0]))

#戻り値はircの結節範囲

In [ ]:
#IRCの結節の範囲
center_irc = xyz2irc(center_xyz,origin_xyz,vxSize_xyz,direction_a)

In [78]:
#CTの画像から指定の範囲をリストで取り出す
def getRawCandidate(self, center_xyz, width_irc):
        center_irc = xyz2irc(
            center_xyz,
            self.origin_xyz,
            self.vxSize_xyz,
            self.direction_a,
        )

        slice_list = []
        # 番号と座標が一軸ごとに取り出される
        中心に幅/2 から　幅分各軸取り出され
        for axis, center_val in enumerate(center_irc):
            start_ndx = int(round(center_val - width_irc[axis]/2))
            end_ndx = int(start_ndx + width_irc[axis])
            slice_list.append(slice(start_ndx, end_ndx))
            
        # ctデータからその範囲を選択して出力

        ct_chunk = ct_a[tuple(slice_list)]

        return ct_chunk, center_irc

## pytorchへ変換

データセットクラスを作成する

標準化と画像のフラット化する

In [ ]:
class Ct:
    def __init__(self, series_uid):
        mhd_path = glob.glob(
            'data-unversioned/part2/luna/subset*/{}.mhd'.format(series_uid)
        )[0]

        ct_mhd = sitk.ReadImage(mhd_path)
        ct_a = np.array(sitk.GetArrayFromImage(ct_mhd), dtype=np.float32)

        # CTs are natively expressed in https://en.wikipedia.org/wiki/Hounsfield_scale
        # HU are scaled oddly, with 0 g/cc (air, approximately) being -1000 and 1 g/cc (water) being 0.
        # The lower bound gets rid of negative density stuff used to indicate out-of-FOV
        # The upper bound nukes any weird hotspots and clamps bone down
        ct_a.clip(-1000, 1000, ct_a)

        self.series_uid = series_uid
        self.hu_a = ct_a

        self.origin_xyz = XyzTuple(*ct_mhd.GetOrigin())
        self.vxSize_xyz = XyzTuple(*ct_mhd.GetSpacing())
        self.direction_a = np.array(ct_mhd.GetDirection()).reshape(3, 3)

    def getRawCandidate(self, center_xyz, width_irc):
        center_irc = xyz2irc(
            center_xyz,
            self.origin_xyz,
            self.vxSize_xyz,
            self.direction_a,
        )

        slice_list = []
        for axis, center_val in enumerate(center_irc):
            start_ndx = int(round(center_val - width_irc[axis]/2))
            end_ndx = int(start_ndx + width_irc[axis])

            assert center_val >= 0 and center_val < self.hu_a.shape[axis], repr([self.series_uid, center_xyz, self.origin_xyz, self.vxSize_xyz, center_irc, axis])

            if start_ndx < 0:
                # log.warning("Crop outside of CT array: {} {}, center:{} shape:{} width:{}".format(
                #     self.series_uid, center_xyz, center_irc, self.hu_a.shape, width_irc))
                start_ndx = 0
                end_ndx = int(width_irc[axis])

            if end_ndx > self.hu_a.shape[axis]:
                # log.warning("Crop outside of CT array: {} {}, center:{} shape:{} width:{}".format(
                #     self.series_uid, center_xyz, center_irc, self.hu_a.shape, width_irc))
                end_ndx = self.hu_a.shape[axis]
                start_ndx = int(self.hu_a.shape[axis] - width_irc[axis])

            slice_list.append(slice(start_ndx, end_ndx))

        ct_chunk = self.hu_a[tuple(slice_list)]

        return ct_chunk, center_irc


In [ ]:
def getCt(series_uid):
    return Ct(series_uid)

In [ ]:
def getCtRawCandidate(series_uid, center_xyz, width_irc):
    ct = getCt(series_uid)
    ct_chunk, center_irc = ct.getRawCandidate(center_xyz, width_irc)
    return ct_chunk, center_irc

In [ ]:
candidateInfo_list[0:5]

データセット作成

In [94]:
from torch.utils.data import Dataset

import torch
import torch.nn as nn
from torchvision import transforms


class LunaDataset(Dataset):
    def __init__(self):
        self.candidateInfo_list = candidateInfo_list
    
    
    
    # ndx:整数を引数にとり(0からN-1),1、結節か否か、2、UID、3、結節の場所（I ,R,C)、4、サンプルテンソルの四つのタプルを返す
    def __getitem__(self, ndx):
        
        
        #注意：中身がない
        
        return (
            candidate_t,
            pos_t,
            candidateInfo_tup.series_uid,
            torch.tensor(center_irc),
        )
        